# Pre-requisites

## Snowflake Open Catalog

Setup an Open Catalog account and create some iceberg tables. To follow along create a catalog named `arctic_tundra`.

- [Getting Started](https://other-docs.snowflake.com/en/opencatalog/tutorials/open-catalog-gs)
- [Doc](https://other-docs.snowflake.com/en/opencatalog/overview)



## Create Open Catalog Integration

**Placeholders**
- `<org>` - Your Snowflake Organization
- `<account>` - Your Snowflake Open Catalog Account
- OAuth Credentials ([Doc](https://other-docs.snowflake.com/en/opencatalog/configure-service-connection))
    - `<client_id>` - OAuth Client Id
    - `<client_secret>` - OAuth Client Secret
- *Optional:*
    - OAUTH_ALLOWED_SCOPES: ([doc](https://other-docs.snowflake.com/en/opencatalog/create-principal-role))

In [ ]:
USE ROLE ACCOUNTADMIN;

CREATE OR REPLACE CATALOG INTEGRATION 
  CATALOG_INT_MY_CATALOG
  CATALOG_SOURCE = POLARIS
  TABLE_FORMAT = ICEBERG
  REST_CONFIG = (
    CATALOG_URI = 'https://<org>-<account>.snowflakecomputing.com/polaris/api/catalog'
    CATALOG_API_TYPE = PUBLIC
    CATALOG_NAME = 'arctic_tundra'
  )
  REST_AUTHENTICATION = (
    TYPE = OAUTH
    OAUTH_CLIENT_ID = '<client_id>'
    OAUTH_CLIENT_SECRET = '<client_secret>'
    OAUTH_ALLOWED_SCOPES = ('PRINCIPAL_ROLE:ALL') -- Or more restrictive scopes
  )
  ENABLED = TRUE;

GRANT USAGE ON INTEGRATION CATALOG_INT_MY_CATALOG TO ROLE SYSADMIN;

## External Volume

In this example we are going to use S3. You will need the details from your Snowflake Open Catalog setup and AWS IAM configuration.

**Placeholders:**
- `<catalog_default_base_location>` - Can be found under Catalog Details > Storage Details
- `<account_id>` - AWS Account Id
- `<role_name>` - AWS Role Name
- `<external_id>` - External Id set in the AWS IAM trust policy


In [ ]:
USE ROLE ACCOUNTADMIN;
CREATE OR REPLACE EXTERNAL VOLUME EXT_VOL_ARCTIC_TUNDRA
   STORAGE_LOCATIONS =
      (
         (
            NAME = 'snowflake-open-catalog-volume'
            STORAGE_PROVIDER = 'S3'
            STORAGE_BASE_URL = '<catalog_default_base_location>'
            STORAGE_AWS_ROLE_ARN = 'arn:aws:iam::<account_id>:role/<role_name>'
            STORAGE_AWS_EXTERNAL_ID = '<external_id>'
         )
      )
      ALLOW_WRITES = TRUE;

USE ROLE SECURITYADMIN;
GRANT USAGE ON EXTERNAL VOLUME EXT_VOL_ARCTIC_TUNDRA TO ROLE SYSADMIN;

## Database
This is the database where your iceberg tables will live, configured to use your Catalog integration.


**Placeholders**
- `<namespace>` - For simplicity sake, we are going to name the schema the same as the namespace in the catalog, but can be whatever you choose.

In [ ]:
USE ROLE SYSADMIN;

CREATE DATABASE IF NOT EXISTS ARCTIC_TUNDRA;
ALTER DATABASE ARCTIC_TUNDRA
  SET CATALOG = 'CATALOG_INT_MY_CATALOG';
  
CREATE SCHEMA ARCTIC_TUNDRA.EXAMPLE;

## Create Stage

Directory settings will be used for the input of a stored procedure at a later step.

**Placeholders:**
- `<catalog_default_base_location>` - Can be found under Catalog Details > Storage Details

In [ ]:
USE ROLE SYSADMIN;

CREATE OR REPLACE STAGE ARCTIC_TUNDRA.EXAMPLE.STG_ARCTIC_TUNDRA
URL = '<catalog_default_base_location>'
STORAGE_INTEGRATION = INT_S3_ARCTIC_TUNDRA
DIRECTORY = (
    ENABLE = true
    AUTO_REFRESH = true
  );

# Stored Procedure
Next we'll create a stored procedure to iterate through the Iceberg tables within the catalog and create the Iceberg tables within Snowflake.

**Parameters:**
- `stage_name` - Stage we created to see iceberg table files
- `schema_name` - Iceberg namespace
- `table_exclusion_pattern` - Wildcard pattern of tables to not create


> The catalog is hardcoded (`CATALOG_INT_MY_CATALOG`) but this could be paramaterized as well



In [ ]:
USE ROLE SYSADMIN;

CREATE OR REPLACE PROCEDURE ARCTIC_TUNDRA.EXAMPLE.SP_SYNC_CATALOG_ICEBERG_TABLES(
    stage_name VARCHAR,
    schema_name VARCHAR,
    table_exclusion_pattern VARCHAR
)
RETURNS STRING
LANGUAGE PYTHON
RUNTIME_VERSION = '3.12'
PACKAGES = ('snowflake-snowpark-python')
HANDLER = 'main'
AS
$$
def main(session, stage_name, schema_name, table_exclusion_pattern):
    
    result_logs = []

    # Dynamic Query
    query = f"""
    WITH iceberg_tables AS (
        SELECT DISTINCT
            CURRENT_DATABASE() AS database_name,
            SPLIT_PART(RELATIVE_PATH, '/', 2) AS schema_name,
            SPLIT_PART(RELATIVE_PATH, '/', 3) AS table_name,
            'CATALOG = ''CATALOG_INT_MY_CATALOG''\\n' ||
            'EXTERNAL_VOLUME = ''EXT_VOL_ARCTIC_TUNDRA''\\n' ||
            'CATALOG_NAMESPACE = ''' || SPLIT_PART(RELATIVE_PATH, '/', 2) || '''\\n' ||
            'CATALOG_TABLE_NAME = ''' || SPLIT_PART(RELATIVE_PATH, '/', 3) || '''' AS statement_params
        FROM DIRECTORY(@{stage_name})
    )
    SELECT
        'CREATE ICEBERG TABLE IF NOT EXISTS ' || 
        UPPER(database_name || '.' || schema_name || '.' || table_name) || '\\n' || 
        statement_params AS stmt
    FROM iceberg_tables
    WHERE schema_name ILIKE '{schema_name}' AND table_name NOT ILIKE '{table_exclusion_pattern}';
    """

    # Execute and iterate over results
    rows = session.sql(query).collect()

    for row in rows:
        stmt = row['STMT']
        try:
            session.sql(stmt).collect()
            result_logs.append(f"Executed: {stmt}")
        except Exception as e:
            result_logs.append(f"Failed: {stmt} -- {str(e)}")

    return "\\n".join(result_logs)
$$;

# Sync

Execute the stored procedure.

In [ ]:
USE ROLE SYSADMIN;
CALL ARCTIC_TUNDRA.EXAMPLE.SP_SYNC_CATALOG_ICEBERG_TABLES('ARCTIC_TUNDRA.EXAMPLE.STG_ARCTIC_TUNDRA','example','sys__%');